In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from CellNet import *

In [2]:
# cuda = True if torch.cuda.is_available() else False

In [3]:

dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))
dataset2 = datasets.MNIST('../data', train=False,transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))

In [4]:
images = []
angles = torch.randint(low=0, high=360, size=(dataset1.data.shape[0],))
for idx, img in enumerate(dataset1.data[:100]):
    samples = []
    angle = angles[idx]
    for i in range(5):
        imgnew = transforms.functional.rotate(img.unsqueeze(0), int(i*angle))
        samples.append(imgnew.squeeze(0))
    images.append(samples)

In [5]:
# num = 0
# print(angles)
# for i in range(5):
#     plt.imshow(images[num][i], cmap='gray')
#     plt.show()

In [33]:
cellnum = 5
incellnum = 2
outcellnum = 1
epochs = 10

In [34]:
cells= []

for _ in range(incellnum):
    cells.append(InCell(64, 64, cellnum-1, 784))

for _ in range(cellnum - incellnum - outcellnum):
    cells.append(Cell(64, 64, cellnum-1))

for _ in range(outcellnum):
    cells.append(OutCell(64, 64, cellnum-1, 1))

# if cuda:
#     for cell in cells:
#         cell.cuda()


optimizers = []

for cell in cells:
    optimizers.append(torch.optim.Adam(cell.parameters()))

In [35]:
def get_n_params_cell(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

params = 0
for cell in cells:
    params += get_n_params_cell(cell)
params

459073

In [36]:
print(cells[0].core)
images[0][0].float().dtype

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], requires_grad=True)


torch.float32

In [28]:
print(cells[0].core)

initialconn = []
for _ in range(cellnum-1):
    initialconn.append(torch.zeros(64))

# if cuda:
#     for idx, i in enumerate(initialconn):
#         initialconn[idx] = i.cuda()

test = images[0][0].float().flatten()



out1, out2 = cells[0](initialconn, test)

# print(cells[0].core)
print(len(out1))
print(out2.shape)

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], requires_grad=True)
7
torch.Size([64])


In [117]:
# Initialize

initialconn = []
for _ in range(cellnum-1):
    initialconn.append(torch.zeros(64))

initialouts = []
for idx, cell in enumerate(cells):
    if idx < incellnum:
        initialouts.append(cell(initialconn, images[0][0].float().flatten()))
    elif idx >= incellnum and idx < cellnum - outcellnum:
        initialouts.append(cell(initialouts))
    else:
        interout, out = cell(initialouts)
        initialouts.append(interout)







for epoch in range(epochs):
    for idx, imgs in enumerate(images):

        

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

IndentationError: expected an indented block after 'for' statement on line 6 (2698001875.py, line 14)